<a href="https://colab.research.google.com/github/arifroska/PrakAPM/blob/main/FP_Growth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
pip install mlxtend --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
#import data
from google.colab import files
uploaded = files.upload()

Saving songs_normalize.csv to songs_normalize (2).csv


In [52]:
import numpy as np
import pandas as pd
import plotly.express as px
import io
from mlxtend.frequent_patterns import fpgrowth

In [53]:
dataset = pd.read_csv(io.BytesIO(uploaded['songs_normalize.csv']), encoding= 'latin-1')
data = dataset[['artist','song','genre']]
print(data)

              artist                                    song             genre
0     Britney Spears                  Oops!...I Did It Again               pop
1          blink-182                    All The Small Things         rock, pop
2         Faith Hill                                 Breathe      pop, country
3           Bon Jovi                            It's My Life       rock, metal
4             *NSYNC                             Bye Bye Bye               pop
...              ...                                     ...               ...
1995  Jonas Brothers                                  Sucker               pop
1996    Taylor Swift                            Cruel Summer               pop
1997    Blanco Brown                              The Git Up  hip hop, country
1998       Sam Smith  Dancing With A Stranger (with Normani)               pop
1999     Post Malone                                 Circles           hip hop

[2000 rows x 3 columns]


In [54]:
data.shape

(2000, 3)

In [55]:
# Gather All Items of Each Song into Numpy Array
song = []
for i in range(0, data.shape[0]):
    for j in range(0, data.shape[1]):
        song.append(data.values[i,j])

# converting to numpy array
song = np.array(song)
print(song)

['Britney Spears' 'Oops!...I Did It Again' 'pop' ... 'Post Malone'
 'Circles' 'hip hop']


In [56]:
#  Transform Them a Pandas DataFrame
df = pd.DataFrame(song, columns=["genre"]) 

# Put 1 to Each Item For Making Countable Table, to be able to perform Group By
df["incident_count"] = 1 

#  Delete NaN Items from Dataset
indexNames = df[df['genre'] == "nan" ].index
df.drop(indexNames , inplace=True)

# Making a New Appropriate Pandas DataFrame for Visualizations  
df_table = df.groupby("genre").sum().sort_values("incident_count", ascending=False).reset_index()

#  Initial Visualizations
df_table.head(5).style.background_gradient(cmap='Blues')

,genre,incident_count
0,pop,428
1,"hip hop, pop",277
2,"hip hop, pop, R&B",244
3,"pop, Dance/Electronic",221
4,"pop, R&B",178


In [57]:
# to have a same origin
df_table["all"] = "Top 50 items" 

# creating tree map using plotly
fig = px.treemap(df_table.head(50), path=['all', "genre"], values='incident_count',
                  color=df_table["incident_count"].head(50), hover_data=['genre'],
                  color_continuous_scale='Blues',
                )
# ploting the treemap
fig.show()

In [58]:
# Transform Every Transaction to Seperate List & Gather Them into Numpy Array
transaction = []
for i in range(df.shape[0]):
    transaction.append([str(df.values[i,j]) for j in range(df.shape[1])])

# creating the numpy array of the transactions
transaction = np.array(transaction)

# importing the required module
from mlxtend.preprocessing import TransactionEncoder

# initializing the transactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
df = pd.DataFrame(te_ary, columns=te.columns_)

# dataset after encoded
df.head()

,#SELFIE - Original Mix,#thatPOWER,'Till I Collapse,(When You Gonna) Give It Up to Me (feat. Keyshia Cole) - Radio Version,*NSYNC,...Ready For It?,03' Bonnie & Clyde,1,1 Thing,"1, 2 Step (feat. Missy Elliott) - Main",...,"rock, pop","rock, pop, Dance/Electronic","rock, pop, metal","rock, pop, metal, Dance/Electronic",rockstar (feat. 21 Savage),set(),t.A.T.u.,we fell in love in october,will.i.am,Ãchame La Culpa
0,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [59]:
# select top 30 items
first30 = df_table["genre"].head(30).values 

# Extract Top 30
df = df.loc[:,first30] 

# shape of the dataset
df.shape

(6000, 30)

In [60]:
#running the fpgrowth algorithm
res=fpgrowth(df,min_support=0.01, use_colnames=True)

# printing top 10
res.head(10)

,support,itemsets
0,0.071333,(pop)
1,0.040667,"(hip hop, pop, R&B)"
2,0.020667,(hip hop)
3,0.029667,"(pop, R&B)"
4,0.036833,"(pop, Dance/Electronic)"
5,0.046167,"(hip hop, pop)"
6,0.013000,"(hip hop, pop, Dance/Electronic)"


In [61]:
# importing required module
from mlxtend.frequent_patterns import association_rules

# creating asssociation rules
nres=association_rules(res, metric="lift", min_threshold=0.01)

# printing association rules
nres

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
